In [13]:
import numpy as np
import pandas as pd

In [14]:
# Load the Excel file again (if needed)
file_path = "CRE_Data_SpringA2025.xlsx"
xls = pd.ExcelFile(file_path)

In [15]:
# Load the Financials sheet
financials_df = pd.read_excel(xls, sheet_name="Financials")

In [16]:
# Clean column names (strip extra spaces)
financials_df.columns = financials_df.columns.str.strip()


In [17]:
# Check the first few rows
print(financials_df.head())

       City  Building ID Class  Year  Gross Rental Income      Lost Rent  \
0  Fairview       365743     B     1         2.065901e+06  152633.686636   
1  Fairview       365743     B     2         2.169196e+06  160265.370968   
2  Fairview       365743     B     3         2.277656e+06  168278.639516   
3  Fairview       365743     B     4         2.391539e+06  176692.571492   
4  Fairview       365743     B     5         2.511116e+06  185527.200067   

   Effective Gross Income  Operating Expenses  Net Operating Income  \
0            1.913268e+06        1.069644e+06          8.436233e+05   
1            2.008931e+06        1.123127e+06          8.858045e+05   
2            2.109378e+06        1.179283e+06          9.300947e+05   
3            2.214846e+06        1.238247e+06          9.765995e+05   
4            2.325589e+06        1.300159e+06          1.025429e+06   

   Vacancy Rate  Operating expense ratio  Rent CAGR  Rent CAGR.1  
0      7.388237                55.906673    0.050

In [19]:
# Compute Mean and Standard Deviation for Risk Metrics
risk_analysis = financials_df.groupby("City").agg(
    avg_noi=("Net Operating Income", "mean"),
    noi_std=("Net Operating Income", "std"),
    avg_vacancy=("Vacancy Rate", "mean"),
    vacancy_std=("Vacancy Rate", "std"),
    avg_op_exp_ratio=("Operating expense ratio", "mean"),
    op_exp_ratio_std=("Operating expense ratio", "std")
).reset_index()

In [20]:
# Compute Coefficient of Variation (Relative Risk)
risk_analysis["NOI_CV"] = risk_analysis["noi_std"] / risk_analysis["avg_noi"]
risk_analysis["Vacancy_CV"] = risk_analysis["vacancy_std"] / risk_analysis["avg_vacancy"]
risk_analysis["Op_Exp_CV"] = risk_analysis["op_exp_ratio_std"] / risk_analysis["avg_op_exp_ratio"]

In [21]:
# Compute Expected Monetary Value (EMV) based on risk exposure
risk_analysis["EMV"] = risk_analysis["avg_noi"] - (0.5 * risk_analysis["noi_std"] + 
                                                    0.3 * risk_analysis["vacancy_std"] + 
                                                    0.2 * risk_analysis["op_exp_ratio_std"])


In [22]:
# Display final risk metrics
print(risk_analysis)

          City       avg_noi        noi_std  avg_vacancy  vacancy_std  \
0     Fairview  9.061587e+05  385123.853317     9.857040     3.000211   
1     New Hope  1.108207e+06  481472.903492    10.177771     2.825514   
2  Springfield  9.623995e+05  413986.981177    10.079560     2.898137   

   avg_op_exp_ratio  op_exp_ratio_std    NOI_CV  Vacancy_CV  Op_Exp_CV  \
0         57.371485          4.255008  0.425007    0.304372   0.074166   
1         50.482743          4.398383  0.434461    0.277616   0.087126   
2         52.012729          4.344397  0.430161    0.287526   0.083526   

             EMV  
0  713594.984206  
1  867468.913995  
2  755404.221234  
